In [1]:
from zad11 import connect, example, load_table, process

In [2]:
spark = connect()

Connecting to Spark...


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/09 20:30:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/06/09 20:30:51 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
load_table(spark, "/input/charts_fmt_small.csv")
load_table(spark, "/input/api_track_to_artist_small.csv")
load_table(spark, "/input/api_track_to_genre_small.csv")
load_table(spark, "/input/charts_daily_sum_small.csv")
load_table(spark, "/input/charts_daily_popularity_small.csv")
load_table(spark, "/input/daily_country_weather_small.csv")
load_table(spark, "/input/wdi_interpolated_small.csv")

Loading '/input/charts_fmt_small.csv' into table 'charts_fmt_small'...


Loading '/input/api_track_to_artist_small.csv' into table 'api_track_to_artist_small'...


Loading '/input/api_track_to_genre_small.csv' into table 'api_track_to_genre_small'...
Loading '/input/charts_daily_sum_small.csv' into table 'charts_daily_sum_small'...
Loading '/input/charts_daily_popularity_small.csv' into table 'charts_daily_popularity_small'...
Loading '/input/daily_country_weather_small.csv' into table 'daily_country_weather_small'...
Loading '/input/wdi_interpolated_small.csv' into table 'wdi_interpolated_small'...


DataFrame[country: string, date: string, rural_population_percent: string, fertility_rate: string, gdp_per_capita_usd: string, mobile_subscriptions_per_100: string, refugee_population_promille: string]

In [4]:
process(spark, "charts_artists_small", """
SELECT
    c.region,
    c.date,
    MD5(a.artist) AS artist_id,
    c.streams
FROM charts_fmt_small AS c
JOIN api_track_to_artist_small AS a
    ON c.track_id = a.track_id
ORDER BY c.region
""")

Processing query and saving to '/charts_artists_small/*'...


+---------+----------+--------------------+-------+
|   region|      date|           artist_id|streams|
+---------+----------+--------------------+-------+
|Argentina|2017-01-01|818ef62dce0d768b9...| 223988|
|Argentina|2017-01-01|8c5b8ea2f957eef56...| 223988|
|Argentina|2017-01-01|6c6e7227e205e6faa...| 210943|
|Argentina|2017-01-01|02b6698efb5bcf3dc...| 173865|
|Argentina|2017-01-01|23a2193a36b72b1eb...| 173865|
|Argentina|2017-01-01|dca432e1624d923de...| 173865|
|Argentina|2017-01-01|82594863c7df01a95...| 173865|
|Argentina|2017-01-01|ed73022e38d784475...| 153956|
|   Poland|2017-01-01|6175b9f6699984f1e...|  25198|
|   Poland|2017-01-01|8ee2c0adee9548498...|  25198|
|   Poland|2017-01-01|309b74a8a61cadf95...|  23163|
|   Poland|2017-01-01|b9fbeb7e58125ff49...|  23163|
|   Poland|2017-01-01|7cc13a578cbab14ed...|  23163|
|   Poland|2017-01-01|3eb0c531dd779dd40...|  24642|
|   Poland|2017-01-01|16fd20c388393f6f0...|  24642|
|   Poland|2017-01-01|ba4c94bc384c116dd...|  24630|
|   Poland|2

In [5]:
process(spark, "charts_genres_small", """
SELECT
    c.region,
    c.date,
    t.genre,
    c.streams
FROM charts_fmt_small AS c
JOIN api_track_to_genre_small AS t
    ON c.track_id = t.track_id
ORDER BY c.region
""")

Processing query and saving to '/charts_genres_small/*'...
+---------+----------+--------------------+-------+
|   region|      date|               genre|streams|
+---------+----------+--------------------+-------+
|Argentina|2017-01-01|           reggaeton| 173865|
|Argentina|2017-01-01|         trap latino| 153956|
|Argentina|2017-01-01|           viral rap| 173865|
|Argentina|2017-01-01|            boy band| 210943|
|Argentina|2017-01-01|           reggaeton| 210943|
|Argentina|2017-01-01|             pop rap| 173865|
|Argentina|2017-01-01|           latin pop| 210943|
|Argentina|2017-01-01|      reggaeton flow| 173865|
|Argentina|2017-01-01|           dance pop| 223988|
|Argentina|2017-01-01|          rap latina| 173865|
|Argentina|2017-01-01|           reggaeton| 223988|
|Argentina|2017-01-01|reggaeton colombiano| 173865|
|Argentina|2017-01-01|    puerto rican pop| 223988|
|Argentina|2017-01-01|                 r&b| 173865|
|Argentina|2017-01-01|           latin pop| 223988|
|Arge

In [6]:
process(spark, "charts_daily_genres_small", """
SELECT
    region,
    date,
    COALESCE(MAX(CASE WHEN genre = 'pop' THEN total_streams END), 0) AS pop,
    COALESCE(MAX(CASE WHEN genre = 'rap' THEN total_streams END), 0) AS rap,
    COALESCE(MAX(CASE WHEN genre = 'rock' THEN total_streams END), 0) AS rock,
    COALESCE(MAX(CASE WHEN genre = 'edm' THEN total_streams END), 0) AS edm,
    COALESCE(MAX(CASE WHEN genre = 'hip hop' THEN total_streams END), 0) AS hip_hop,
    COALESCE(MAX(CASE WHEN genre = 'trap latino' THEN total_streams END), 0) AS trap_latino,
    COALESCE(MAX(CASE WHEN genre = 'reggaeton' THEN total_streams END), 0) AS reggaeton,
    COALESCE(MAX(CASE WHEN genre = 'electropop' THEN total_streams END), 0) AS electropop,
    COALESCE(MAX(CASE WHEN genre = 'dance pop' THEN total_streams END), 0) AS dance_pop,
    COALESCE(MAX(CASE WHEN genre = 'pop rap' THEN total_streams END), 0) AS pop_rap,
    COALESCE(MAX(CASE WHEN genre = 'musica mexicana' THEN total_streams END), 0) AS musica_mexicana,
    COALESCE(MAX(CASE WHEN genre = 'trap' THEN total_streams END), 0) AS trap,
    COALESCE(MAX(CASE WHEN genre = 'modern rock' THEN total_streams END), 0) AS modern_rock,
    COALESCE(MAX(CASE WHEN genre = 'classic rock' THEN total_streams END), 0) AS classic_rock,
    COALESCE(MAX(CASE WHEN genre = 'uk pop' THEN total_streams END), 0) AS uk_pop,
    COALESCE(MAX(CASE WHEN genre = 'k-pop' THEN total_streams END), 0) AS k_pop,
    COALESCE(MAX(CASE WHEN genre = 'tropical house' THEN total_streams END), 0) AS tropical_house,
    COALESCE(MAX(CASE WHEN genre = 'melodic rap' THEN total_streams END), 0) AS melodic_rap,
    COALESCE(MAX(CASE WHEN genre = 'canadian pop' THEN total_streams END), 0) AS canadian_pop,
    COALESCE(MAX(CASE WHEN genre = 'modern bollywood' THEN total_streams END), 0) AS modern_bollywood
FROM (
    SELECT
        region,
        date,
        genre,
        SUM(streams) AS total_streams
    FROM charts_genres_small
    WHERE genre IN ('pop', 'rap', 'rock', 'edm', 'hip hop', 'trap latino', 'reggaeton', 'electropop', 'dance pop', 'pop rap', 'musica mexicana', 'trap', 'modern rock', 'classic rock', 'uk pop', 'k-pop', 'tropical house', 'melodic rap', 'canadian pop', 'modern bollywood')
    GROUP BY region, date, genre
)
GROUP BY region, date
ORDER BY region, date
""")

Processing query and saving to '/charts_daily_genres_small/*'...


+---------+----------+-------+--------+----+-------+--------+-----------+---------+----------+---------+--------+---------------+----+-----------+------------+-------+-----+--------------+-----------+------------+----------------+
|   region|      date|    pop|     rap|rock|    edm| hip_hop|trap_latino|reggaeton|electropop|dance_pop| pop_rap|musica_mexicana|trap|modern_rock|classic_rock| uk_pop|k_pop|tropical_house|melodic_rap|canadian_pop|modern_bollywood|
+---------+----------+-------+--------+----+-------+--------+-----------+---------+----------+---------+--------+---------------+----+-----------+------------+-------+-----+--------------+-----------+------------+----------------+
|Argentina|2017-01-01|    0.0|173865.0| 0.0|    0.0|173865.0|   551809.0| 762752.0|       0.0| 223988.0|173865.0|            0.0| 0.0|        0.0|         0.0|    0.0|  0.0|           0.0|        0.0|         0.0|             0.0|
|   Poland|2017-01-01|99293.0|     0.0| 0.0|74095.0|     0.0|        0.0|   

In [7]:
process(spark, "charts_genre_popularity_small", """
SELECT
    g.region,
    g.date,
    g.pop / s.streams AS pop,
    g.rap / s.streams AS rap,
    g.rock / s.streams AS rock,
    g.edm / s.streams AS edm,
    g.hip_hop / s.streams AS hip_hop,
    g.trap_latino / s.streams AS trap_latino,
    g.reggaeton / s.streams AS reggaeton,
    g.electropop / s.streams AS electropop,
    g.dance_pop / s.streams AS dance_pop,
    g.pop_rap / s.streams AS pop_rap,
    g.musica_mexicana / s.streams AS musica_mexicana,
    g.trap / s.streams AS trap,
    g.modern_rock / s.streams AS modern_rock,
    g.classic_rock / s.streams AS classic_rock,
    g.uk_pop / s.streams AS uk_pop,
    g.k_pop / s.streams AS k_pop,
    g.tropical_house / s.streams AS tropical_house,
    g.melodic_rap / s.streams AS melodic_rap,
    g.canadian_pop / s.streams AS canadian_pop,
    g.modern_bollywood / s.streams AS modern_bollywood
FROM charts_daily_genres_small AS g
JOIN charts_daily_sum_small AS s
    ON g.region = s.region AND g.date = s.date
""")

Processing query and saving to '/charts_genre_popularity_small/*'...


+---------+----------+------------------+-------------------+----+------------------+-------------------+------------------+------------------+-------------------+-------------------+-------------------+---------------+----+-----------+------------+-------------------+-----+------------------+-----------+-------------------+----------------+
|   region|      date|               pop|                rap|rock|               edm|            hip_hop|       trap_latino|         reggaeton|         electropop|          dance_pop|            pop_rap|musica_mexicana|trap|modern_rock|classic_rock|             uk_pop|k_pop|    tropical_house|melodic_rap|       canadian_pop|modern_bollywood|
+---------+----------+------------------+-------------------+----+------------------+-------------------+------------------+------------------+-------------------+-------------------+-------------------+---------------+----+-----------+------------+-------------------+-----+------------------+-----------+------

In [8]:
process(spark, "output_small", """
SELECT
    cgp.region AS country,
    cgp.date,
    cds.streams AS daily_streams,
    cdp.popularity AS daily_popularity,
    dcw.temperature_c AS temperature_c,
    dcw.precipitation_mm AS precipitation_mm,
    wdi.rural_population_percent AS `wdi:rural_population_percent`,
    wdi.fertility_rate AS `wdi:fertility_rate`,
    wdi.gdp_per_capita_usd AS `wdi:gdp_per_capita_usd`,
    wdi.mobile_subscriptions_per_100 AS `wdi:mobile_subscriptions_per_100`,
    wdi.refugee_population_promille AS `wdi:refugee_population_promille`,
    cgp.pop AS `genre:pop`,
    cgp.rap AS `genre:rap`,
    cgp.rock AS `genre:rock`,
    cgp.edm AS `genre:edm`,
    cgp.hip_hop AS `genre:hip_hop`,
    cgp.trap_latino AS `genre:trap_latino`,
    cgp.reggaeton AS `genre:reggaeton`,
    cgp.electropop AS `genre:electropop`,
    cgp.dance_pop AS `genre:dance_pop`,
    cgp.pop_rap AS `genre:pop_rap`,
    cgp.musica_mexicana AS `genre:musica_mexicana`,
    cgp.trap AS `genre:trap`,
    cgp.modern_rock AS `genre:modern_rock`,
    cgp.classic_rock AS `genre:classic_rock`,
    cgp.uk_pop AS `genre:uk_pop`,
    cgp.k_pop AS `genre:k_pop`,
    cgp.tropical_house AS `genre:tropical_house`,
    cgp.melodic_rap AS `genre:melodic_rap`,
    cgp.canadian_pop AS `genre:canadian_pop`,
    cgp.modern_bollywood AS `genre:modern_bollywood`
FROM charts_genre_popularity_small AS cgp
JOIN charts_daily_popularity_small AS cdp
    ON cgp.region = cdp.region AND cgp.date = cdp.date
JOIN charts_daily_sum_small AS cds
    ON cgp.region = cds.region AND cgp.date = cds.date
JOIN daily_country_weather_small AS dcw
    ON cgp.region = dcw.country AND cgp.date = dcw.date
JOIN wdi_interpolated_small AS wdi
    ON cgp.region = wdi.country AND cgp.date = wdi.date
ORDER BY cgp.region, cgp.date
""")

Processing query and saving to '/output_small/*'...


25/06/09 20:32:25 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------+----------+-------------+----------------+------------------+----------------+----------------------------+------------------+----------------------+--------------------------------+-------------------------------+------------------+-------------------+----------+------------------+-------------------+------------------+------------------+-------------------+-------------------+-------------------+---------------------+----------+-----------------+------------------+-------------------+-----------+--------------------+-----------------+-------------------+----------------------+
|  country|      date|daily_streams|daily_popularity|     temperature_c|precipitation_mm|wdi:rural_population_percent|wdi:fertility_rate|wdi:gdp_per_capita_usd|wdi:mobile_subscriptions_per_100|wdi:refugee_population_promille|         genre:pop|          genre:rap|genre:rock|         genre:edm|      genre:hip_hop| genre:trap_latino|   genre:reggaeton|   genre:electropop|    genre:dance_pop|      genre

In [9]:
example(spark, "charts_fmt_small")
example(spark, "api_track_to_artist_small")
example(spark, "charts_artists_small")
example(spark, "api_track_to_genre_small")
example(spark, "charts_genres_small")
example(spark, "charts_daily_genres_small")
example(spark, "charts_daily_sum_small")
example(spark, "charts_genre_popularity_small")
example(spark, "charts_daily_popularity_small")
example(spark, "daily_country_weather_small")
example(spark, "wdi_interpolated_small")
example(spark, "output_small")

+---------+----------+----------------------+-------+
|region   |date      |track_id              |streams|
+---------+----------+----------------------+-------+
|Argentina|2017-01-01|6mICuAdrwEjh6Y6lroV2Kg|253019 |
|Argentina|2017-01-01|7DM4BPaS7uofFul3ywMe46|223988 |
|Argentina|2017-01-01|3AEZUABDXNtecAOSC1qTfo|210943 |
|Argentina|2017-01-01|6rQSrBHf7HlZjtcMZ4S4bO|173865 |
|Argentina|2017-01-01|58IL315gMSTD37DOZPJ2hf|153956 |
|Poland   |2017-01-01|4pdPtRcBmOSQDlJ3Fk945m|26290  |
|Poland   |2017-01-01|5aAx2yezTd8zXrkmtKl66Z|25198  |
|Poland   |2017-01-01|7BKLCZ1jbUBVqRi2FVlTVw|24642  |
|Poland   |2017-01-01|7abpmGpF7PGep2rDU68GBR|24630  |
|Poland   |2017-01-01|5knuzwU65gJK7IF5yJsuaW|23163  |
+---------+----------+----------------------+-------+

+----------------------+-----------------+
|track_id              |artist           |
+----------------------+-----------------+
|5aAx2yezTd8zXrkmtKl66Z|The Weeknd       |
|5aAx2yezTd8zXrkmtKl66Z|Daft Punk        |
|7DM4BPaS7uofFul3ywMe46|Rick

+---------+----------+-------------+----------------+------------------+----------------+----------------------------+------------------+----------------------+--------------------------------+-------------------------------+------------------+-------------------+----------+------------------+-------------------+------------------+------------------+-------------------+-------------------+-------------------+---------------------+----------+-----------------+------------------+-------------------+-----------+--------------------+-----------------+-------------------+----------------------+
|country  |date      |daily_streams|daily_popularity|temperature_c     |precipitation_mm|wdi:rural_population_percent|wdi:fertility_rate|wdi:gdp_per_capita_usd|wdi:mobile_subscriptions_per_100|wdi:refugee_population_promille|genre:pop         |genre:rap          |genre:rock|genre:edm         |genre:hip_hop      |genre:trap_latino |genre:reggaeton   |genre:electropop   |genre:dance_pop    |genre:pop_r

In [10]:
spark.stop()